## Applied Data Science Capstone Notebook*
---
\* this notebook will be mainly used for the capstone project

In [1]:
import pandas as pd
import numpy as np

In [2]:
print('Hello Capstone Project Course!')

Hello Capstone Project Course!


## Segmenting and Clustering Neighborhoods in Toronto

Getting URL with a list of neighborhoods in Toronto.

In [3]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

Using `requests` library to download web page

In [4]:
import requests
data = requests.get(url).text

Using `BeautifulSoup` to parse HTML

In [5]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(data,"html5lib")

Using sample code from [Hints for scraping Notebook](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/NewLinkWebscrapingHints.md) to scrape HTML and create dataframe

In [6]:
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        
        table_contents.append(cell)

#print(table_contents)

In [7]:
df=pd.DataFrame(table_contents)

In [8]:
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

In [9]:
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


Moving `PostalCode` column on first place

In [10]:
fixed_columns = [df.columns[-1]] + list(df.columns[:-1])
df = df[fixed_columns]
df.head()

,Neighborhood,PostalCode,Borough
0,Parkwoods,M3A,North York
1,Victoria Village,M4A,North York
2,"Regent Park, Harbourfront",M5A,Downtown Toronto
3,"Lawrence Manor, Lawrence Heights",M6A,North York
4,Ontario Provincial Government,M7A,Queen's Park


In [11]:
print("Number of rows: {}".format(df.shape[0]))

Number of rows: 103


Getting geographical coordinates of each postal code provided in csv file

In [12]:
geo_spatial_csv = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv'

In [13]:
geo_df = pd.read_csv(geo_spatial_csv)
geo_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Renaming `Postal Code` in `geo_df` to match `PostalCode` in `df` to merge them later

In [14]:
geo_df.rename(columns={'Postal Code':'PostalCode'}, inplace=True)
geo_df.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Merging two dataframes on `PostalCode`

In [15]:
df = df.merge(geo_df, on="PostalCode", how = 'inner')
df.head()

,Neighborhood,PostalCode,Borough,Latitude,Longitude
0,Parkwoods,M3A,North York,43.753259,-79.329656
1,Victoria Village,M4A,North York,43.725882,-79.315572
2,"Regent Park, Harbourfront",M5A,Downtown Toronto,43.654260,-79.360636
3,"Lawrence Manor, Lawrence Heights",M6A,North York,43.718518,-79.464763
4,Ontario Provincial Government,M7A,Queen's Park,43.662301,-79.389494


Counting `PostalCode` in each `Borough`

In [16]:
df['Borough'].value_counts()

North York                24
Scarborough               17
Downtown Toronto          17
Etobicoke                 11
Central Toronto            9
West Toronto               6
York                       5
East Toronto               4
East York                  4
Queen's Park               1
East York/East Toronto     1
Downtown Toronto Stn A     1
Mississauga                1
East Toronto Business      1
Etobicoke Northwest        1
Name: Borough, dtype: int64

Analysing each `Borough` by setting `1` for `PostalCode` in corresponding column

In [17]:
df_onehot = pd.get_dummies(df[['Borough']], prefix="", prefix_sep="")
df_onehot['PostalCode'] = df['PostalCode']
fixed_columns = [df_onehot.columns[-1]] + list(df_onehot.columns[:-1])
df_onehot = df_onehot[fixed_columns]
df_onehot.head()

,PostalCode,Central Toronto,Downtown Toronto,Downtown Toronto Stn A,East Toronto,East Toronto Business,East York,East York/East Toronto,Etobicoke,Etobicoke Northwest,Mississauga,North York,Queen's Park,Scarborough,West Toronto,York
0,M3A,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
1,M4A,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
2,M5A,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
3,M6A,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
4,M7A,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0


In [18]:
df_onehot.shape

(103, 16)

Run k-means to cluster the neighborhood into 5 clusters.

In [19]:
from sklearn.cluster import KMeans

In [20]:
kclusters = 5

df_clustering = df_onehot.drop('PostalCode', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(df_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([2, 2, 3, 2, 0, 4, 1, 2, 0, 3], dtype=int32)

Adding cluster to dataframe

In [21]:
df_onehot.insert(0, 'Cluster Labels', kmeans.labels_)
df_onehot.head()

,Cluster Labels,PostalCode,Central Toronto,Downtown Toronto,Downtown Toronto Stn A,East Toronto,East Toronto Business,East York,East York/East Toronto,Etobicoke,Etobicoke Northwest,Mississauga,North York,Queen's Park,Scarborough,West Toronto,York
0,2,M3A,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
1,2,M4A,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
2,3,M5A,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
3,2,M6A,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
4,0,M7A,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0


Merging with initial data frame

In [22]:
df = df.merge(df_onehot[['Cluster Labels','PostalCode']], on="PostalCode", how = 'inner')
df.head()

,Neighborhood,PostalCode,Borough,Latitude,Longitude,Cluster Labels
0,Parkwoods,M3A,North York,43.753259,-79.329656,2
1,Victoria Village,M4A,North York,43.725882,-79.315572,2
2,"Regent Park, Harbourfront",M5A,Downtown Toronto,43.654260,-79.360636,3
3,"Lawrence Manor, Lawrence Heights",M6A,North York,43.718518,-79.464763,2
4,Ontario Provincial Government,M7A,Queen's Park,43.662301,-79.389494,0


Examine `Downtown Toronto` cluster

In [23]:
df.loc[df['Cluster Labels'] == 4,  df.columns[:-1]]

,Neighborhood,PostalCode,Borough,Latitude,Longitude
5,Islington Avenue,M9A,Etobicoke,43.667856,-79.532242
11,"West Deane Park, Princess Gardens, Martin Grov...",M9B,Etobicoke,43.650943,-79.554724
17,"Eringate, Bloordale Gardens, Old Burnhamthorpe...",M9C,Etobicoke,43.643515,-79.577201
70,Westmount,M9P,Etobicoke,43.696319,-79.532242
77,"Kingsview Village, St. Phillips, Martin Grove ...",M9R,Etobicoke,43.688905,-79.554724
88,"New Toronto, Mimico South, Humber Bay Shores",M8V,Etobicoke,43.605647,-79.501321
89,"South Steeles, Silverstone, Humbergate, Jamest...",M9V,Etobicoke,43.739416,-79.588437
93,"Alderwood, Long Branch",M8W,Etobicoke,43.602414,-79.543484
98,"The Kingsway, Montgomery Road, Old Mill North",M8X,Etobicoke,43.653654,-79.506944
101,"Old Mill South, King's Mill Park, Sunnylea, Hu...",M8Y,Etobicoke,43.636258,-79.498509


Examine `Etobicoke` cluster

In [24]:
df.loc[df['Cluster Labels'] == 3,  df.columns[:-1]]

,Neighborhood,PostalCode,Borough,Latitude,Longitude
2,"Regent Park, Harbourfront",M5A,Downtown Toronto,43.654260,-79.360636
9,"Garden District, Ryerson",M5B,Downtown Toronto,43.657162,-79.378937
15,St. James Town,M5C,Downtown Toronto,43.651494,-79.375418
20,Berczy Park,M5E,Downtown Toronto,43.644771,-79.373306
24,Central Bay Street,M5G,Downtown Toronto,43.657952,-79.387383
25,Christie,M6G,Downtown Toronto,43.669542,-79.422564
30,"Richmond, Adelaide, King",M5H,Downtown Toronto,43.650571,-79.384568
36,"Harbourfront East, Union Station, Toronto Islands",M5J,Downtown Toronto,43.640816,-79.381752
42,"Toronto Dominion Centre, Design Exchange",M5K,Downtown Toronto,43.647177,-79.381576
48,"Commerce Court, Victoria Hotel",M5L,Downtown Toronto,43.648198,-79.379817


Examine `North York` cluster

In [25]:
df.loc[df['Cluster Labels'] == 2,  df.columns[:-1]]

,Neighborhood,PostalCode,Borough,Latitude,Longitude
0,Parkwoods,M3A,North York,43.753259,-79.329656
1,Victoria Village,M4A,North York,43.725882,-79.315572
3,"Lawrence Manor, Lawrence Heights",M6A,North York,43.718518,-79.464763
7,Don Mills North,M3B,North York,43.745906,-79.352188
10,Glencairn,M6B,North York,43.709577,-79.445073
13,Don Mills South,M3C,North York,43.725900,-79.340923
27,Hillcrest Village,M2H,North York,43.803762,-79.363452
28,"Bathurst Manor, Wilson Heights, Downsview North",M3H,North York,43.754328,-79.442259
33,"Fairview, Henry Farm, Oriole",M2J,North York,43.778517,-79.346556
34,"Northwood Park, York University",M3J,North York,43.767980,-79.487262


Examine `Scarborough` cluster

In [26]:
df.loc[df['Cluster Labels'] == 1,  df.columns[:-1]]

,Neighborhood,PostalCode,Borough,Latitude,Longitude
6,"Malvern, Rouge",M1B,Scarborough,43.806686,-79.194353
12,"Rouge Hill, Port Union, Highland Creek",M1C,Scarborough,43.784535,-79.160497
18,"Guildwood, Morningside, West Hill",M1E,Scarborough,43.763573,-79.188711
22,Woburn,M1G,Scarborough,43.770992,-79.216917
26,Cedarbrae,M1H,Scarborough,43.773136,-79.239476
32,Scarborough Village,M1J,Scarborough,43.744734,-79.239476
38,"Kennedy Park, Ionview, East Birchmount Park",M1K,Scarborough,43.727929,-79.262029
44,"Golden Mile, Clairlea, Oakridge",M1L,Scarborough,43.711112,-79.284577
51,"Cliffside, Cliffcrest, Scarborough Village West",M1M,Scarborough,43.716316,-79.239476
58,"Birch Cliff, Cliffside West",M1N,Scarborough,43.692657,-79.264848


Examine cluster with other borough

In [27]:
df.loc[df['Cluster Labels'] == 0,  df.columns[:-1]]

,Neighborhood,PostalCode,Borough,Latitude,Longitude
4,Ontario Provincial Government,M7A,Queen's Park,43.662301,-79.389494
8,"Parkview Hill, Woodbine Gardens",M4B,East York,43.706397,-79.309937
14,Woodbine Heights,M4C,East York,43.695344,-79.318389
16,Humewood-Cedarvale,M6C,York,43.693781,-79.428191
19,The Beaches,M4E,East Toronto,43.676357,-79.293031
21,Caledonia-Fairbanks,M6E,York,43.689026,-79.453512
23,Leaside,M4G,East York,43.709060,-79.363452
29,Thorncliffe Park,M4H,East York,43.705369,-79.349372
31,"Dufferin, Dovercourt Village",M6H,West Toronto,43.669005,-79.442259
35,The Danforth East,M4J,East York/East Toronto,43.685347,-79.338106


Create map with Borough clusters

In [28]:
#!conda install -c conda-forge folium=0.5.0 --yes 

In [29]:
import folium 
import matplotlib.cm as cm
import matplotlib.colors as colors

In [30]:
# create map
map_clusters = folium.Map(location=[43.70011, -79.4163], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df['Latitude'], df['Longitude'], df['Neighborhood'], df['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

The only observation I can make is that only Scarborough Borough has Postal codes stating with M1